In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
from funcs import read_config, get_groups, read_dataset, get_kpi2_respuestas, get_kpi3_retencion, get_kpi4_influencia

In [2]:
#Data Pull and Functions
data_frames = read_dataset() 
states= data_frames.get('1_states.parquet')
categories = data_frames.get('2_categories.parquet')
user_yelp = data_frames.get('3_user_yelp.parquet')
user_google = data_frames.get('4_user_google.parquet')
business_google = data_frames.get('5_business_google.parquet')
business_yelp = data_frames.get('6_business_yelp.parquet')
categories_google = data_frames.get('7_categories_google.parquet')
categories_yelp = data_frames.get('8_categories_yelp.parquet')
reviews_google = data_frames.get('9_reviews_google.parquet')
reviews_yelp = data_frames.get('10_reviews_yelp.parquet')
groups_google = data_frames.get('11_grupo_de_categorias_google.parquet')
groups_yelp = data_frames.get('12_grupo_de_categorias_yelp.parquet')
df_user = data_frames.get('user_categories')
df_categories = data_frames.get('locales_categories')

2023-12-11 22:01:05.579 
  command:

    streamlit run /home/p/Code/Henry/PI_DA/.env/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2023-12-11 22:01:05.581 No runtime found, using MemoryCacheStorageManager


1_states.parquet: (4, 2)
2_categories.parquet: (1297, 2)
3_user_yelp.parquet: (353245, 6)
4_user_google.parquet: (1360619, 5)
5_business_google.parquet: (24755, 6)
6_business_yelp.parquet: (12917, 6)
7_categories_google.parquet: (70307, 2)
8_categories_yelp.parquet: (61869, 2)
9_reviews_google.parquet: (2430142, 8)
10_reviews_yelp.parquet: (909986, 6)
11_grupo_de_categorias_google.parquet: (29183, 2)
12_grupo_de_categorias_yelp.parquet: (17819, 2)
user_categories: (1713864, 3)
locales_categories: (132176, 4)


In [3]:
# Definir las listas de valores únicos
unique_groups = groups_google['group'].unique()
unique_states = states['state'].unique().tolist()
reviews_google['date'] = pd.to_datetime(reviews_google['date'])
unique_years = reviews_google['date'].dt.year.unique()
unique_categories = get_groups(categories)
target_objetive = 5

# Obtén las listas de combinaciones para el bucle
combinations = [(group, state, year) for group in unique_groups for state in unique_states for year in unique_years]


In [4]:

# KPI 2
def get_kpi2_respuestas(reviews_google, business_google, categories_groups, state, target_state, target_group, target_year, target_objetive):
    '''
    Calcula la calidad de las respuestas
    kpi_respuestas = (ratio respuestas/review * rango tiempo) + sentiment_resp promedio

    target_state(lista de str) = lista de estados elegidos por el usuario 
    target_group(lista de str) = lista de grupos de categorias elegidas por el usuario
    target_year(lista de enteros) = lista de años elegidos para su analsis 
    target_objetive(float) = objetivo de aumento para aplicar a la variable 'kpi2_valor'
    '''
    # Elegimos el id del estado
    id_estado_elegidos = state[state['state'].apply(lambda x: x in target_state)]['state_id'].to_list()

    # Filtramos los negocios por el estado elegido
    business_google = business_google[business_google['state_id'].apply(lambda x: x in id_estado_elegidos)]

    # Filtramos por grupo de categorías
    categories_groups = categories_groups[categories_groups['group'].apply(lambda x: x in target_group)]

    # Filtramos los restaurantes por categoría
    business_google = pd.merge(business_google, categories_groups[['gmap_id']], on='gmap_id')

    # Filtramos las reviews
    reviews_google = pd.merge(reviews_google, business_google[['gmap_id']], on='gmap_id')

    # Cambiamos el tipo de dato de las fechas
    reviews_google['date'] = pd.to_datetime(reviews_google['date'])
    reviews_google['resp_date'] = pd.to_datetime(reviews_google['resp_date'])

    # Filtramos por año
    reviews_google = reviews_google[reviews_google['date'].dt.year.apply(lambda x: x in target_year)]

    # Vemos los que tienen respuesta
    try:
        con_respuesta = reviews_google['resp_sentiment'].value_counts().drop('0.0').sum()
    except KeyError:
        con_respuesta = len(reviews_google)

    # Promedio de cuántas tienen respuesta
    try:
        ratio_resp_rev = con_respuesta / len(reviews_google) ########################
    except ZeroDivisionError:
        return print('No existe el año pedido')

    # Diferencia de tiempo
    rango_de_tiempo = (reviews_google[reviews_google['resp_sentiment'] != '0.0']['resp_date'] - reviews_google[
        reviews_google['resp_sentiment'] != '0.0']['date']).dt.total_seconds() / 3600
    rango_de_tiempo = rango_de_tiempo.apply(lambda x: abs(x)) ####################

    # Sacamos el promedio del resp_sentiment
    reviews_google['resp_sentiment'] = reviews_google['resp_sentiment'].astype(float)
    promedio_resp_sent = reviews_google[reviews_google['resp_sentiment'] != 0.0]['resp_sentiment'].mean() #######################

    # Calcular el objetivo aplicado solo a kpi2_valor
    kpi2_valor = (ratio_resp_rev * rango_de_tiempo.mean() / 100) + promedio_resp_sent
    objetivo_kpi2_valor = round((kpi2_valor * (1 + target_objetive / 100)), 2)
    
    # Crear el diccionario con los resultados
    kpi2 = {
        'kpi2_valor': round(kpi2_valor, 2),
        'ratio_resp_rev': round(ratio_resp_rev, 2),
        'rango_de_tiempo': round(rango_de_tiempo.mean(), 2),
        'promedio_resp_sent': round(promedio_resp_sent, 2),
        'objetivo_kpi2_valor': round(objetivo_kpi2_valor, 2)
    }

    return kpi2    

In [12]:
# Bucle for para calcular el KPI 2 para diferentes combinaciones
for target_group, target_state, target_year in combinations:
    # Llama a la función para obtener el KPI 2
    kpi2_result = get_kpi2_respuestas(reviews_google, business_google, groups_google, states,
                                      [target_state], [target_group], [target_year], target_objetive)
    print(f"Resultados para {target_group}, {target_state}, {target_year}")
    print(kpi2_result)
    print("---") 

Resultados para general, Florida, 2021
{'kpi2_valor': 0.79, 'ratio_resp_rev': 0.13, 'rango_de_tiempo': 345.93, 'promedio_resp_sent': 0.34, 'objetivo_kpi2_valor': 0.83}
---
Resultados para general, Florida, 2020
{'kpi2_valor': 0.91, 'ratio_resp_rev': 0.12, 'rango_de_tiempo': 466.02, 'promedio_resp_sent': 0.33, 'objetivo_kpi2_valor': 0.96}
---
Resultados para general, Florida, 2019
{'kpi2_valor': 0.77, 'ratio_resp_rev': 0.09, 'rango_de_tiempo': 525.2, 'promedio_resp_sent': 0.3, 'objetivo_kpi2_valor': 0.81}
---
Resultados para general, Florida, 2017
{'kpi2_valor': 1.37, 'ratio_resp_rev': 0.04, 'rango_de_tiempo': 2281.03, 'promedio_resp_sent': 0.36, 'objetivo_kpi2_valor': 1.44}
---
Resultados para general, Florida, 2018
{'kpi2_valor': 0.89, 'ratio_resp_rev': 0.06, 'rango_de_tiempo': 845.86, 'promedio_resp_sent': 0.37, 'objetivo_kpi2_valor': 0.94}
---
Resultados para general, Florida, 2016
{'kpi2_valor': 1.85, 'ratio_resp_rev': 0.03, 'rango_de_tiempo': 4377.47, 'promedio_resp_sent': 0.38, '

In [6]:
print(f"Resultados para {target_group}, {target_state}, {target_year}")
print(kpi2_result)
print("---") 

Resultados para veggie & vegetarian, New Jersey, 2015
{'kpi2_valor': 7.65, 'ratio_resp_rev': 0.05, 'rango_de_tiempo': 15334.69, 'promedio_resp_sent': 0.46, 'objetivo_kpi2_valor': 8.03}
---


In [7]:
states

,state_id,state
0,1,Florida
1,2,California
2,3,Illinois
3,4,New Jersey


In [8]:
reviews_google

,review_id,user_id,gmap_id,stars,sentiment,date,resp_sentiment,resp_date
0,0,1.0899115226265579e+20,0x80c2c778e3b73d33:0xbdc58662a4a97d49,5,2,2021-01-06 05:12:07,0.0,2021-01-06 05:12:07
1,1,1.1129032221979622e+20,0x80c2c778e3b73d33:0xbdc58662a4a97d49,5,2,2021-02-09 05:47:28,0.0,2021-02-09 05:47:28
2,2,1.1264035744961195e+20,0x80c2c778e3b73d33:0xbdc58662a4a97d49,4,0,2020-03-08 05:04:42,0.0,2020-03-08 05:04:42
3,3,1.1744034972382365e+20,0x80c2c778e3b73d33:0xbdc58662a4a97d49,5,2,2019-03-07 05:56:56,0.0,2019-03-07 05:56:56
4,4,1.0058077083612353e+20,0x80c2c778e3b73d33:0xbdc58662a4a97d49,5,2,2017-05-16 05:01:41,0.0,2017-05-16 05:01:41
...,...,...,...,...,...,...,...,...
9599941,2452767,1.0910540659763497e+20,0x880fce33edd16c7b:0x44e066027b82ad8d,4,0,2016-09-01 01:03:33,0.0,2016-09-01 01:03:33
9599942,2452768,1.1598048245262603e+20,0x880fce33edd16c7b:0x44e066027b82ad8d,4,0,2017-07-26 19:09:52,0.0,2017-07-26 19:09:52
9599943,2452769,1.159471047965541e+20,0x880fce33edd16c7b:0x44e066027b82ad8d,4,0,2015-11-16 17:06:38,0.0,2015-11-16 17:06:38
9599944,2452770,1.1792358760866297e+20,0x880fce33edd16c7b:0x44e066027b82ad8d,3,0,2016-10-06 10:41:33,0.0,2016-10-06 10:41:33


In [9]:
unique_groups = groups_google['group'].unique()
print(unique_groups) 

['general' 'fast food' 'bars & nightlife' 'foreign' 'coffess & breakfast'
 'veggie & vegetarian']


In [10]:
df = pd.read_parquet('../../datasets/processed/bd/5_business_google.parquet.gz')
df[df['name']=='The Hummus Corner of Wekiva']

,gmap_id,name,latitude,longitude,avg_stars,state_id
39,0x88e7746b43fd35fb:0xcdf54b983cf4df07,The Hummus Corner of Wekiva,28.70645,-81.445392,4.4,1


In [11]:
df = pd.read_parquet('../../datasets/processed/bd/6_business_yelp.parquet.gz')
df[df['name']=='The Hummus Corner of Wekiva']

,business_id,name,latitude,longitude,avg_stars,state_id
